In [1]:
#이전 실습은 별도의 은닉층없이 신경망을 구성해서 확인을 해본 것이다.
#우리가 신경망을 딥하게 가져가면 (은닉층을 다양하게 구성) 좀 더 정확도 높은 모델이 생성이 되어질 것임
#항상 딥하게 은닉층을 가져간다고 해서 100% 좋은 정확도를 가지는 것은 아님 따라서 실습을 통해 확인해보도록 할 것임.

from tensorflow import keras #keras로 다이렉트 접근이 가능함

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [2]:
print(train_input.shape, train_target.shape, test_input.shape, test_target.shape) #데이터가 제대로 담겨져 있는지 확인

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [3]:
from sklearn.model_selection import train_test_split

#훈련용 데이터에서 검증용데이터를 따로 뽑아내도록 한다.
#먼저, 데이터를 정규화해주도록 한다.
train_scaled = train_input / 255.
train_scaled = train_scaled.reshape(-1, 28*28) #1차원으로 펼쳐주면서 입력데이터로 활용하도록 한다.

train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=1234)
#훈련데이터를 8:2(훈련과 검증)로 분리해서 인덱스 값으로 반환해주게 될 것임

In [4]:
print(train_scaled.shape, train_target.shape, val_scaled.shape, val_target.shape)

(48000, 784) (48000,) (12000, 784) (12000,)


In [5]:
# 신경망 만들기
# layers(층)을 추가하는 방법1
dense1 = keras.layers.Dense(units=100, input_shape=(28 * 28,), activation='sigmoid') #첫번째 은닉층의 뉴런의 갯수 100개로 출력
#최종 출력단만 softmax로 지정을 해주면 된다. / 그전까지의 각각의 은닉층은 sigmoid로 지정
dense2 = keras.layers.Dense(units=10, activation='softmax') #추가적인 은닉층, 바로 출력단으로 연결 (1개만 추가하도록 함)

In [6]:
model = keras.Sequential([dense1, dense2]) #기존에는 model에 add하면서 추가를 해줬었는데 리스트를 통해서 신경망에 세포들을 넣어주도록 한다.

model.summary()
#Param = 784(28*28) * 100(출력) + 100(바이어스값) = 78500
# 78400이 W의 값이고 바이어스계층이 100개임
#이를 출력단에 연결!! 출력이 10개일때 w의 갯수는 100 * 10 + 바이어스값 10개 = 1010

#단지 은닉층 하나만 추가했을뿐인데 79,400으로 독립적인 w의 갯수를 각각 현재 입력데이터에 대해서 계산을 해주게 되는 것임

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [10]:
# layers(층)을 추가하는 방법2 - 다이렉트로 은닉층을 넣어줄 수 있음. +) dense마다 이름을 부여해줄 수 있음

model = keras.Sequential([ #신경망 생성과 동시에 dense를 다이렉트로 전달해줄 수 있음
    keras.layers.Dense(units=100, input_shape=(28 * 28,), activation='sigmoid', name='hidden'), # 은닉층이라는 의미 hidden의 이름 부여
    keras.layers.Dense(units=10, activation='softmax', name='output') #출력단이라는 이름을 부여
], name='패션 MNIST 모델') 

model.summary() #모델에 대한 정보 출력

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd', metrics='accuracy')
# 데이터값 자체가 정수값으로 담겨져 있으면 그것을 원핫인코딩으로 펼쳐줘야 하는 것임
# 원핫인코딩을 함수를 이용해서 펼쳐주는게 귀찮다면 sparse_의 키워드를 붙여주면서 자동적으로 펼쳐주도록 한다.

In [8]:
model.fit(train_scaled, train_target, epochs=100) 

Epoch 1/100
1500/1500 [==============================] - 2s 1ms/step - loss: 1.4085 - accuracy: 0.6350
Epoch 2/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.8444 - accuracy: 0.7428
Epoch 3/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.7098 - accuracy: 0.7637
Epoch 4/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6443 - accuracy: 0.7809
Epoch 5/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6024 - accuracy: 0.7940
Epoch 6/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5719 - accuracy: 0.8032
Epoch 7/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5486 - accuracy: 0.8106
Epoch 8/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5304 - accuracy: 0.8171
Epoch 9/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5153 - accuracy: 0.8222
Epoch 10/100
1500/1500 [==============================] - 2s 1ms/step - l

1500/1500 [==============================] - 2s 1ms/step - loss: 0.3397 - accuracy: 0.8803
Epoch 80/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3386 - accuracy: 0.8806
Epoch 81/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3377 - accuracy: 0.8815
Epoch 82/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3370 - accuracy: 0.8810
Epoch 83/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3362 - accuracy: 0.8815
Epoch 84/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3353 - accuracy: 0.8824
Epoch 85/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3346 - accuracy: 0.8821
Epoch 86/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3339 - accuracy: 0.8831
Epoch 87/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3327 - accuracy: 0.8830
Epoch 88/100
1500/1500 [==============================] - 2s 1ms/step - loss:

In [9]:
model.evaluate(val_scaled, val_target) #검증용으로 뽑아둔 데이터를 통해 정확도를 확인
#단지 은닉층을 하나 추가한 것 뿐인데도 거의 2%의 정확도가 높아진 것을 확인할 수 있다.

375/375 [==============================] - 1s 959us/step - loss: 0.3638 - accuracy: 0.8678


[0.36375653743743896, 0.8678333163261414]

In [ ]:
#손실함수까지의 개념을 살펴본 것임.
#미니배치 학습 개념이 소개되고 있음 
# => 별도로 셋팅을 하지 않으면 fit하여 학습을 진행할 때 미니학습으로 학습이 되어지게끔 디폴트처리가 되어지고 있음
#빠른 결과를 피드백 받아볼 수 있음!!

#미니배치의 개념
#패션 이미지 데이터셋은 실질적으로 학습을 할 때마다 6만개의 모든 데이터를 가지고 학습을 시킨다고 하면 시간이 어마어마하게 걸릴 수 있음
#임의의 100장을 뽑아서 한 에포크마다 수행을 할 수 있게끔 한다.
#한번 훈련에 적용되는 전체 이미지중에 학습에 사용할 뽑아낸 100장의 데이터를 미니배치라고 표현을 해준다.
#훈련데이터 중에서 일부만 랜덤하게 골라서 학습할 때마다 에포크 수행시 처리되는 방법이다.

#미니배치의 손실함수 계측을 통해서 전체 훈련데이터의 근사치를 이용
#이렇게 학습을 시키더라도 전체 데이터로 학습시킨것과 별 차이가 없다는 것이 현재까지 검증된 바이다.

#결과의 큰 차이가 없다고 한다면 학습시간을 단축시켜서 빠르게 모델을 생성할 수 있는 처리를 일반적으로 수행하게끔 하는 것임
#따라서 디폴트도 미니배치를 수행하도록 텐서플로우나 사이킷런에 설정이 되어져 있음

#손실함수를 통해서 w값에서 접선의 기울기를 구해서 접선의 기울기값에 기반해서 
#다음 스탭을 이동해가면서 최종목표는 0에 근접하도록 하여 잔차의 값을 최소화하도록 한다. => 접선의 기울기를 구할 때 미분을 하면됨

# 기울기라는 것은 두 점을 지나는 직선이다. 기울기는 어떻게 구할 수 있을까? x의 증가량분의 y의 증가량! 
# 그때의 x증가량을 h라고 한다면 x분의 f(x+h) - f(x)임 => 이에 대한 계산을 미분이라고 한다. 따라서 f(x)를 말함
# h값을 0에 근접시키면 접선의 기울기가 나올 것이다. 

#두점이 지나가는 폭을 0에 가깝게 수렴시키면 접선의 기울기, 접선의 기울기가 미분의 정의가 됨


#입력 핏쳐가 두 개를 가지는 x0, x1 데이터 셋에서 신경망을 구성한다고 할 때 
#손실함수에서는 각 잔차에서의 오차값을 최소화하는 대표값으로 mse, crossentropy을 활용
#다항식은 편미분을 이용해서 각각에 편미분에 의한 오차에 최소값을 찾아가는 수학적인 방법을 활용할 수 있다.
# 편미분이란 x0를 기준으로 잡으면 그 나머지의 값들은 상수화되어지는 개념임
# x0의 편미분을 구하라고 한다면 미분은 2x의 지수 -1을 해주면 됨. 상수를 미분하면 0이 됨. 편미분을 하게 되면 2x1임
# 잔차의 최소의 값을 구하는 방법은 각각의 편미분에 의한 잔차의 값을 구하는 것이다.
#문제발생) 그물망의 특징을 띄는 표를 위에서 바라보게 되면 가운데로 쏠리게 될 것임. 
#          이때, 방향을 가진 벡터의 형태로 그려지게 됨. 이때의 기울기는 함수에 가장 낮은 최소값을 가리키게 될 것이다.
#                가장 낮은 곳에서 멀어질수록 화살표의 크기가 커지게 되는 특징을 가지게 됨
#                기울기가 가리키는 곳은 출력의 값을 줄여가는 쪽으로 방향이 가져가게 됨

#실제로 일일히 편미분을 구해서 w값을 업데이트하게끔 학습을 하느냐. 그렇지 않다.
# 우리가 데이터를 저장할 때 중복을 허용하지 않는다면 저장하는 순간 기존 저장된 데이터인지를 먼저 검색을 해봐야함
# 그러다보니 일일히 비교하는 방식으로 저장을 하게 된다면 문제없이 처리되어지지만 저장된 데이터의 양이 많아질수록 그만큼의 저장속도가 떨어짐
# 효율적으로 데이터의 중복을 빠르게 처리할지에 대한 부분이 중복 데이터에 대한 허용을 하지 않는 기능에서의 핵심 아이디어, 알고리즘임
# 3차원 공간을 벗어나는 특징이 다양하다면 한번에 보여주는 것이 불가능

#=> 모든 변수의 편미분을 동시에 계산하고 싶다면 양쪽의 편미분을 묶어서 계산하도록 한다. 선형대수의 연산을 수행하면 됨

#경사하강법 알고리즘을 통해서 적용을 해서 최종적인 잔차의 값이 최소화되어지는 순간에 최적의 w값을 찾아보는 것
#경사하강법의 값은 어떻게 변화되는가. 현재 가중치값에서 한번에 학습이 일어난 순간에 미분을 구해보고 구한 미분의 러닝레잇을 빼줌
# 접선의 기울기를 구하면서 얼마만큼으로 이동해가고, 지금 담겨진 w의 값을 업데이트 해주는 과정을 에폭시만큼 반복하면서 최상의 값을 뽑아내도록 한다.

#신경망에서의 기울기: w가 2행 3열(망을 구성할 때 입력과 출력이 연결된 신경망이라고 가정)이라면 !!입력데이터는 두가지 핏쳐를 가지고 있고 출력은 3개로 되어진다고 생각할 수 있음!! 
#입력으로 학습하고 있는 데이터셋에 대해서 최적의 w값을 학습하는 것이 목표 -> 딥러닝

#[확률적 경사 하강법] -> 데이터를 미니배치로 무작위로 선정하여 경사하강법으로 매개변수를 갱신(각각의 가중치(w)를 의미)
# 신경망 같은 경우 학습절차: 가중치와 편향을 훈련데이터에 적응하도록 조정 -> 미니배치 -> 기울기 산출 -> 매개변수 갱신 -> 반복(잔차값이 더이상 줄어들지 않을 때까지)

#오차역전파법 (해시의 개념을 적용한 내용임_검색을 빠르게 할 수 있음.)
#실질적인 신경망에서의 그 수많은 각각의 w값들을 어떻게 학습이 이루어지면서 업데이트되어져 가는지를 실제 구현한 알고리즘임
#내부에 손실함수와 경사하강법의 알고리즘이 실제로는 라이브러리안에 어떻게 구현이 되어져 있는지, 우리는 알고리즘을 들여다보면서 파악

#계산그래프 : 연산기를 따로 빼놓고 분류 연산을 진행 
# 오차역전파법이라는 알고리즘에 핵심은 지금부터 시작임 
# 입력의 핏쳐가 있고, 디폴트 w(랜덤값에서 시작)에 곱셈하여 출력을 내보내도록 함 부호를 뉴런으로 생각 
#-> ***최종적으로 도출되는 값이 신경망의 예측값임*** 

#전통알고리즘에서는 예측값과 정답값에서 w가 어떻게 업데이트를 하느냐. (최적화된 w를 찾는 것이 최종 목표)
#분류군일때는 cross entropy error로 계산. 하나의 대표값이 나오는데 최소가 되어지는 값 도출해야 함
#w는 랜덤값에서 학습이 끝나고 나면 w - 학습률 * 접선의기울기(미분구하기) => x1에 대한 x의 편미분을 구함. => 점점 정답에 가까워지게끔 업데이트가 되어질 것임
#그래서 loss값이 최소가 되어지는 값이 학습에 대한 결과로 보여줌. 

#역방향을 역전파(계산하는 방법이 에러율_잔차값을 계산하는 것임) / 방향 그대로 가는 것을 순전파
# 역전파에서 최종적으로 1의 값을 두고 봤을 때 가격에 최종 결과값을 결정하게 되는 
# 주요포인트는 내 원래 단가에서 지불해야할 금액이 나오게 되면 가중치값에 의해서 최종단가가 결정됨
# 최종결과는 2.2배의 결과치가 지불해야할 금액인데 1을 보면서 2.2를 계산하는게 쉽지 않음.
# 단계를 보면서 계산을 해주게 되면 지불할 돈을 보면서 관계성을 고려하여 출력한다면 계산상 수월해짐

#입력이 200인데 출력이 220이 되어야 한다고 하면 곱셈연산기는 내 입력값에 1.1을 곱해줘야하는데 그 값이 입력으로 들어가는 비중값임
#계산기와 입력값을 핏쳐로 생각, 게이트를 생각하면 x1과 x2가 있을 때 

#1차함수와 시그모이드를 통과시킨 값이 동일하다고 생각 (값이 커지면 왜곡되어지는 선형을 최소화하기 위함) => 결과적으로 로그값이 나옴
#오차값이 적어지는 단계값을 도출하는 것이 최종적인 목표

#핏쳐의 값은 고정이 되어져 있고 결과적으로 예측할 수 있는 비중에 대한 값도 고정이 되어야 한다는 의미가 포함되어져 있음
# 그 고정된 값이 사과에 대한 예제에서는 가중치로 1.1이라고 할 수 있음
#즉, 이에 대한 관계에서 1.1을 거꾸로 도출해내기 위해서는 학습레잇에 대해 미분을 곱해서 빼면 됨!! 미분 계산이 중요
#입력값이 뭐가 전달되는지 알고 출력값이 뭔지 알면 얼마만큼의 비중이 나와야하는지 계산돼서 나옴 -> 이 계산은 미분이라는 것임